## This notebook runs 1000 Monte Carlo simulations

In the earlier pipeline stage `05_exploratory_plots`, we plotted quantities such as $P_{gv}(k)$, but didn't assign error bars, since we used a limited number ($\sim$ 5) of Monte Carlos. Next, we run 1000 Monte Carlos, in order to make these plots with less statistical noise, and with proper error bars. The logic is split between this pipeline stage (`06_run_sims`) and the next stage (`07_the_end`).

#### How to run this notebook

 - In principle, you can start this notebook, walk away for $\sim 5$ hours, and end up with 1000 Monte Carlo simulations.

 - In practice, I prefer not to do it this way, since it requires babysitting the connection to the jupyterlab server for $\sim 5$ hours.

 - Here's what I do: log in with ssh, convert the jupyter notebook (`06_run_sims.ipynb`) to a python script (`06_run_sims.py`), and run the script from the command line. (As a final detail, I run the script in a screen/tmux session, so that I can disconnect the ssh session.)
   
   ```
   jupyter nbconvert --to python 06_run_sims.ipynb
   python 06_run_sims.py  # will run for ~5 hours  
   ```

#### Output files

 - `pk_data.npy`: contains power spectra $P_{gg}$, $P_{gv}$, $P_{vv}$ of SDSS data, for two velocity reconstructions (90 and 150 GHz). File format is a single `(nfields,nfields,nkbins)` numpy array, where the `nfields` axes have length 3 and corrresponds to fields {gal, v90, v150}.

 - `pk_mocks.npy`: contains power spectra $P_{gg}$, $P_{gv}$, $P_{vv}$ of SDSS **mocks**. File format is a single `(nmocks,nfields,nfields,nkbins)` numpy array, where the `nfields` axes have length 3 and corrresponds to fields {gal, v90, v150}.

 - `pk_surrogates.npy`: contains power spectra $P_{gg}$, $P_{gv}$, $P_{vv}$ of SDSS **surrogates**. File format is a single `(nsurr,nfnl,nfields,nfields,nkbins)` numpy array. The `nfnl` axis has length 3 and corresponds to $f_{NL} = (-250,0,250)$. The `nfields` axes have length 3 and corrrespond to fields {gal, v90, v150}.

## Imports and global variables

In [1]:
import os
import time
import kszx
import functools
import numpy as np

[fig:340924] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.fig.1000/jf.0/3433889792/shared_mem_cuda_pool.fig could be created.
[fig:340924] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


In [2]:
# File 'global_params.py' in current directory
import global_params

nkbins = global_params.nkbins
kbin_edges = global_params.kbin_edges
kbin_centers = global_params.kbin_centers

surr_bg = global_params.surr_bg
surr_bv = global_params.surr_bv
surr_fnl = global_params.surr_fnl

nmocks = global_params.sdss_nmocks
nsurr = global_params.num_surrogates

## Read input files
  - `bounding_box.pkl`: created in pipeline stage `04_prepare_catalogs.ipynb`
  - `ngal.npy`: created in pipeline stage `04_prepare_catalogs.ipynb`

In [3]:
box = kszx.io_utils.read_pickle('bounding_box.pkl')
cosmo = kszx.Cosmology('planck18+bao')

mock_ngal = kszx.io_utils.read_npy('mock_ngal.npy')
ngal_mean = np.mean(mock_ngal)
ngal_rms = np.var(mock_ngal)**0.5

Reading bounding_box.pkl
Running CAMB
Reading mock_ngal.npy


## Setting up power spectrum estimation

These functions are very similar to their counterparts in `05_exploratory_plots.ipynb`.

In [4]:
@functools.cache
def get_pse(mrand):
    """Returns a kszx.KszPSE object. 
    
    A KszPSE computes power spectra involving a galaxy field and one or more velocity
    reconstructions, or surrogates for these fields. In our pipeline, we use two velocity
    reconstructions vr_90, vr_150 (corresponding to 90+150 GHz CMB data). See the KszPSE
    docstring for more info.
    
    If 'mrand' is True, use the mock random catalog, instead of the random catalog.
    (The KszPSE constructor needs the random catalog to define the survey geometry.)
    """

    print(f'Constructing PSE ({mrand=})')
    
    rname = 'mock_randoms.h5' if mrand else 'randoms.h5'
    catalog = kszx.Catalog.from_h5(f'catalogs/{rname}')
    
    return kszx.KszPSE(
        box = box, 
        cosmo = cosmo, 
        randcat = catalog, 
        kbin_edges = kbin_edges,
        surr_ngal_mean = ngal_mean,   # ngal_mean was computed above
        surr_ngal_rms = ngal_rms,     # ngal_rms was computed above
        surr_bg = surr_bg,
        surr_bv = surr_bv,
        rweights = catalog.wfkp,
        nksz = 2,
        # ksz_rweights = None,
        ksz_bv = [ catalog.bv_90, catalog.bv_150 ],
        ksz_tcmb_realization = [ catalog.tcmb_90, catalog.tcmb_150 ]
    )

In [5]:
def run_data():
    """Compute P_gg, P_gv, and P_vv for SDSS data, and cache the result in 'pk_data.npy'.
    
    The result is returned as a shape (nfields, nfields, nkbins) array, where the field index
    has length nfields=3, and indexes a field { gal, vr_90, vr_150 }. Thus, the returned array
    contains all auto and cross power spectra involving the galaxy field and velocity reconstructions.
    """
    
    filename = 'pk_data.npy'
    if os.path.exists(filename):
        time.sleep(0.1)  # work around race condition in python multiprocessing
        return kszx.io_utils.read_npy(filename)

    gcat = kszx.Catalog.from_h5('catalogs/galaxies.h5')
    pse = get_pse(False)

    pk = pse.eval_pk(
        gcat = gcat,
        gweights = gcat.wfkp * (gcat.wzf + gcat.wcp - 1) * gcat.wsys,
        # ksz_gweights = None, 
        ksz_bv = [ gcat.bv_90, gcat.bv_150 ], 
        ksz_tcmb = [ gcat.tcmb_90, gcat.tcmb_150 ]
    )

    kszx.io_utils.write_npy(filename, pk)
    return pk


In [6]:
def run_mock(i):
    """Compute P_gg, P_gv, and P_vv for one SDSS mock, and cache the result in power_spectra/pk_mock_{i}.npy.

    The result is returned as a shape (nfields, nfields, nkbins) array, where the field index
    has length nfields=3, and indexes a field { gal, vr_90, vr_150 }. Thus, the returned array
    contains all auto and cross power spectra involving the galaxy field and velocity reconstructions.
    """
    
    filename = f'power_spectra/pk_mock_{i}.npy'
    if os.path.exists(filename):
        time.sleep(0.1)  # work around race condition in python multiprocessing
        return kszx.io_utils.read_npy(filename)

    print(f'Generating {filename}\n', end='')
    gcat = kszx.Catalog.from_h5(f'catalogs/mock_{i}.h5')
    pse = get_pse(True)
    
    pk = pse.eval_pk(
        gcat = gcat,
        gweights = gcat.wfkp,  # no sysweights for mocks
        # ksz_gweights = None, 
        ksz_bv = [ gcat.bv_90, gcat.bv_150 ], 
        ksz_tcmb = [ gcat.tcmb_90, gcat.tcmb_150 ]
    )
        
    kszx.io_utils.write_npy(filename, pk)
    return pk

In [7]:
def run_surrogate(i):
    """Compute P_gg, P_gv, and P_vv for one SDSS surrogate (and for fNL=-250,0,250), and cache the result in power_spectra/pk_surr_{i}.npy.
    
    The result is retuend as a shape (nfnl, nfields, nfields, nkbins) array.
    The fnl index has length nfnl=3, and runs over fnl = { -250, 0, 250 }.
    The field index has length nfields=3, and runs over { gal, vr_90, vr_150 }.
    """
    
    filename = f'power_spectra/pk_surr_{i}.npy'
    if os.path.exists(filename):
        time.sleep(0.1)  # work around race condition in python multiprocessing
        return kszx.io_utils.read_npy(filename)

    print(f'Generating {filename}\n', end='')
    pse = get_pse(False)
    
    pse.simulate_surrogate(enable_fnl = True)
    pk = pse.eval_pk_surrogate(fnl=[-surr_fnl, 0, surr_fnl])    
    
    kszx.io_utils.write_npy(filename, pk)
    return pk

## Run everything

  - We use a multiprocessing Pool to run realizations in parallel, and use all CPU cores.

  - **IMPORTANT NOTE**: we construct the power spectrum estimators before creating the multiprocessing Pool. This is okay because the estimators are the same for all workers in the pool. If instead of doing this, we constructed a power spectrum estimator independently on each worker, the memory usage would be much higher!

  - **IMPORANT NOTE**: we use a kszx.utils.Pool, not a multiprocessing.Pool. This reseeds the global numpy RNG in each worker process. If we didn't do this, there would be a lot of duplicate surrogate sims! See the [kszx.utils.Pool](https://kszx.readthedocs.io/en/latest/misc_utils.html#kszx.utils.Pool) docstring for more discussion.

In [8]:
# IMPORTANT NOTE: we construct the power spectrum estimators before creating the multiprocessing Pool.
# This is okay because the estimators are the same for all workers in the pool.

# If instead of doing this, we constructed a power spectrum estimator independently on each worker, the
# memory usage would be much higher!

def construct_pse_if_file_does_not_exist(filename, mrand):
    if not os.path.exists(filename):
        get_pse(mrand)

construct_pse_if_file_does_not_exist('pk_data.npy', mrand=False)

for i in range(nmocks):
    construct_pse_if_file_does_not_exist(f'power_spectra/pk_mock_{i}.npy', mrand=True)

for i in range(nsurr):
    construct_pse_if_file_does_not_exist(f'power_spectra/pk_surr_{i}.npy', mrand=False)

In [9]:
# IMPORTANT NOTE: we use a kszx.utils.Pool, not a multiprocessing.Pool.
# This reseeds the global numpy RNG in each worker process.
# If we didn't do this, there would be a lot of duplicate surrogate sims.

with kszx.utils.Pool() as pool:
    # Submit jobs to queue, return multiprocessing.AsyncResult objects.
    pk_data = pool.apply_async(run_data)
    pk_mocks = pool.map_async(run_mock, range(nmocks), chunksize=1)
    pk_surrogates = pool.map_async(run_surrogate, range(nsurr), chunksize=1)

    # Wait on results and convert to numpy arrays.
    pk_data = pk_data.get()                        # shape (3,3,nkbins), already written to pk_data.npy
    pk_mocks = np.array(pk_mocks.get())            # shape (nmocks,3,3,nkbins)
    pk_surrogates = np.array(pk_surrogates.get())  # shape (nmocks,3,3,3,nkbins)

Reading pk_data.npy
Reading power_spectra/pk_mock_0.npy
Reading power_spectra/pk_mock_1.npy
Reading power_spectra/pk_mock_3.npy
Reading power_spectra/pk_mock_4.npy
Reading power_spectra/pk_mock_5.npy
Reading power_spectra/pk_mock_2.npy
Reading power_spectra/pk_mock_6.npy
Reading power_spectra/pk_mock_8.npy
Reading power_spectra/pk_mock_9.npy
Reading power_spectra/pk_mock_10.npy
Reading power_spectra/pk_mock_7.npy
Reading power_spectra/pk_mock_11.npy
Reading power_spectra/pk_mock_12.npy
Reading power_spectra/pk_mock_13.npy
Reading power_spectra/pk_mock_14.npy
Reading power_spectra/pk_mock_16.npy
Reading power_spectra/pk_mock_17.npy
Reading power_spectra/pk_mock_15.npy
Reading power_spectra/pk_mock_18.npy
Reading power_spectra/pk_mock_23.npy
Reading power_spectra/pk_mock_22.npy
Reading power_spectra/pk_mock_21.npy
Reading power_spectra/pk_mock_27.npy
Reading power_spectra/pk_mock_26.npy
Reading power_spectra/pk_mock_24.npy
Reading power_spectra/pk_mock_20.npy
Reading power_spectra/pk_moc

In [10]:
# Just checking!
assert pk_data.shape == (3,3,nkbins)
assert pk_mocks.shape == (nmocks,3,3,nkbins)
assert pk_surrogates.shape == (nsurr,3,3,3,nkbins)

kszx.io_utils.write_npy('pk_mocks.npy', pk_mocks)
kszx.io_utils.write_npy('pk_surrogates.npy', pk_surrogates)    

Writing pk_mocks.npy
Writing pk_surrogates.npy
